In [1]:
import pandas as pd
import json

# GBIF dataset - Get current accepted names and taxonIDs

Use option `dtype=str` to avoid interpreting taxonIDs as numeric

In [2]:
# Read GBIF Backbone taxonomy
gbif_backbone = pd.read_table(
    "resources/gbif_backbone/Taxon.tsv",
    sep="\t",
    dtype=str,
    na_values=None
)

# acceptedNameUsageID is blank for currently accepted taxa
# logically this should be equal to taxonID - makes some downstream processing less convoluted
gbif_backbone.loc[gbif_backbone.taxonomicStatus == 'accepted', 'acceptedNameUsageID'] = gbif_backbone.loc[gbif_backbone.taxonomicStatus == 'accepted', 'taxonID']

In [3]:
# Raw dataset
dataset = pd.read_table(
    "data/0097412-230224095556074.csv",
    sep="\t",
    dtype=str,
    na_values=None
)

In [4]:
dataset.head()

,taxonKey,scientificName,acceptedTaxonKey,acceptedScientificName,numberOfOccurrences,taxonRank,taxonomicStatus,kingdom,kingdomKey,phylum,...,classKey,order,orderKey,family,familyKey,genus,genusKey,species,speciesKey,iucnRedListCategory
0,2722299,Carex ×pieperiana Junge,2722300,Carex ×ruedtii Kneuck.,2,SPECIES,SYNONYM,Plantae,6,Tracheophyta,...,196,Poales,1369,Cyperaceae,7708,Carex,2721893,Carex ruedtii,2722300,NE
1,2874569,Cucumis sativus L.,2874569,Cucumis sativus L.,33,SPECIES,ACCEPTED,Plantae,6,Tracheophyta,...,220,Cucurbitales,7224005,Cucurbitaceae,6634,Cucumis,2874568,Cucumis sativus,2874569,NE
2,2888292,Pyrola media Sw.,2888292,Pyrola media Sw.,249,SPECIES,ACCEPTED,Plantae,6,Tracheophyta,...,220,Ericales,1353,Ericaceae,2505,Pyrola,2888249,Pyrola media,2888292,NE
3,2996064,Rubus rhytidophyllus H.E.Weber,2996064,Rubus rhytidophyllus H.E.Weber,50,SPECIES,ACCEPTED,Plantae,6,Tracheophyta,...,220,Rosales,691,Rosaceae,5015,Rubus,2988638,Rubus rhytidophyllus,2996064,NE
4,2998929,Rubus kiesewetteri Henker,2998929,Rubus kiesewetteri Henker,16,SPECIES,ACCEPTED,Plantae,6,Tracheophyta,...,220,Rosales,691,Rosaceae,5015,Rubus,2988638,Rubus kiesewetteri,2998929,NE


In [5]:
dataset.value_counts('kingdom')

kingdom
Plantae           7207
Animalia             1
incertae sedis       1
Name: count, dtype: int64

In [6]:
dataset.query('kingdom != "Plantae"')

,taxonKey,scientificName,acceptedTaxonKey,acceptedScientificName,numberOfOccurrences,taxonRank,taxonomicStatus,kingdom,kingdomKey,phylum,...,classKey,order,orderKey,family,familyKey,genus,genusKey,species,speciesKey,iucnRedListCategory
1722,0,incertae sedis,NaN,NaN,19304,KINGDOM,NaN,incertae sedis,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3161,1346141,"Ammophila W.Kirby, 1798",1346141,"Ammophila W.Kirby, 1798",485,GENUS,ACCEPTED,Animalia,1,Arthropoda,...,216,Hymenoptera,1457,Sphecidae,4352,Ammophila,1346141,NaN,NaN,NaN


In [70]:
# Merge on taxonID
dataset_merged = pd.merge(
    left=dataset.query("taxonRank == 'SPECIES'")[['taxonKey','scientificName']],
    right=gbif_backbone[[
        'taxonID',
        'parentNameUsageID',
        'scientificName',
        'scientificNameAuthorship',
        'canonicalName',
        'acceptedNameUsageID',
        'taxonomicStatus',
        'kingdom','phylum','class','order','family','genus'
    ]],
    how='left',
    left_on = 'taxonKey',
    right_on = 'taxonID'
).rename(
    columns={
        'scientificName_y' : 'scientificName',
        'taxonKey' : 'dataset_taxonID',
        'scientificName_x' : 'dataset_scientificName'
    }
)

In [71]:
# Get accepted scientific name
dataset_merged = pd.merge(
    left=dataset_merged,
    right=gbif_backbone[['taxonID','scientificName']].rename(columns={'taxonID' : 'acceptedNameUsageID', 'scientificName' : 'acceptedName'}),
    left_on="acceptedNameUsageID",
    right_on="acceptedNameUsageID",
    how="left"
)

In [72]:
# NaN: taxonID not present in the current GBIF backbone taxonomy
# Doubtful: no accepted name in current GBIF backbone taxonomy
dataset_merged.value_counts('taxonomicStatus', dropna=False)

taxonomicStatus
accepted               4873
synonym                 396
homotypic synonym       367
doubtful                105
heterotypic synonym      85
NaN                      50
Name: count, dtype: int64

In [73]:
# Drop doubtful and missing taxa
dataset_merged_curr = dataset_merged[[not i for i in dataset_merged.acceptedNameUsageID.isna()]]

In [74]:
dataset_merged_curr.value_counts('class')

class
Magnoliopsida     4534
Liliopsida        1013
Polypodiopsida     102
Pinopsida           57
Lycopodiopsida      14
Ginkgoopsida         1
Name: count, dtype: int64

In [75]:
dataset_merged_curr[['scientificName','family','taxonID']].to_csv("dataset_merged_curr.tsv", sep="\t", index=False, quoting=None)

In [76]:
dataset_merged_curr.head()

,dataset_taxonID,dataset_scientificName,taxonID,parentNameUsageID,scientificName,scientificNameAuthorship,canonicalName,acceptedNameUsageID,taxonomicStatus,kingdom,phylum,class,order,family,genus,acceptedName
0,2722299,Carex ×pieperiana Junge,2722299,2721893,Carex pieperiana Junge,Junge,Carex pieperiana,2722300,synonym,Plantae,Tracheophyta,Liliopsida,Poales,Cyperaceae,Carex,Carex ruedtii Kneuck.
1,2874569,Cucumis sativus L.,2874569,2874568,Cucumis sativus L.,L.,Cucumis sativus,2874569,accepted,Plantae,Tracheophyta,Magnoliopsida,Cucurbitales,Cucurbitaceae,Cucumis,Cucumis sativus L.
2,2888292,Pyrola media Sw.,2888292,2888249,Pyrola media Sw.,Sw.,Pyrola media,2888292,accepted,Plantae,Tracheophyta,Magnoliopsida,Ericales,Ericaceae,Pyrola,Pyrola media Sw.
3,2996064,Rubus rhytidophyllus H.E.Weber,2996064,2988638,Rubus rhytidophyllus H.E.Weber,H.E.Weber,Rubus rhytidophyllus,2996064,accepted,Plantae,Tracheophyta,Magnoliopsida,Rosales,Rosaceae,Rubus,Rubus rhytidophyllus H.E.Weber
4,2998929,Rubus kiesewetteri Henker,2998929,2988638,Rubus kiesewetteri Henker,Henker,Rubus kiesewetteri,2998929,accepted,Plantae,Tracheophyta,Magnoliopsida,Rosales,Rosaceae,Rubus,Rubus kiesewetteri Henker


# NCBI Taxonomy - get species level taxa within Viridiplantae

In [13]:
shortlist = {}
# Faster to use taxidlineage from new_taxdump than to compute lineages locally
with open('resources/ncbi_taxonomy/taxidlineage.dmp', 'r') as fh_in:
    for line in fh_in:
        spl = line.split("\t|\t")
        if len(spl) > 1:
            lineage = spl[1].split(" ")
            if "33090" in lineage: # Viridplantae taxonID
                shortlist[spl[0]] = { 'lineage' : lineage }
with open('resources/ncbi_taxonomy/nodes.dmp', 'r') as fh_in:
    for line in fh_in:
        spl = line.split("\t|\t")
        if len(spl) > 3:
            if spl[0] in shortlist and spl[2] == "species":
                shortlist[spl[0]]['rank'] = 'species'
with open('resources/ncbi_taxonomy/names.dmp', 'r') as fh_in:
    for line in fh_in:
        spl = line.removesuffix("\t|\n").split("\t|\t")
        if spl[0] in shortlist and 'rank' in shortlist[spl[0]] and shortlist[spl[0]]['rank'] == 'species':
            if spl[3] == 'scientific name': # Should only be one
                shortlist[spl[0]]['canonicalName'] = spl[1]
            elif spl[3] == 'authority': # there can be more than one 'authority' for a given taxonID because synonyms share same NCBI taxonID
                if 'scientificName' not in shortlist[spl[0]]:
                    shortlist[spl[0]]['scientificName'] = [spl[1]]
                else:
                    shortlist[spl[0]]['scientificName'].append(spl[1])
with open('ncbi_viridiplantae.tsv', 'w') as fh_out:
    fh_out.write("scientificName\ttaxonID\n") # Header line
    for taxid in shortlist:
        if 'rank' in shortlist[taxid] and shortlist[taxid]['rank'] == 'species':
            if 'scientificName' in shortlist[taxid]:
                for name in shortlist[taxid]['scientificName']:
                    fh_out.write("\t".join([name, taxid]))
                    fh_out.write("\n")
            elif 'canonicalName' in shortlist[taxid]:
                fh_out.write("\t".join([shortlist[taxid]['canonicalName'], taxid]))
                fh_out.write("\n")
            else:
                print("Warning: no scientificName or canonicalName for taxon " + taxid)

In [14]:
%%bash
head ncbi_viridiplantae.tsv

scientificName	taxonID
Chloroparvula japonica Lopes dos Santos, Noel & Eikrem, 2017	1411623
Chloroparvula pacifica Lopes dos Santos, Noel & Eikrem, 2017	1883388
Chloroparvula cf. pacifica RCC999	464255
Chloroparvula sp.	2850713
Chloroparvula sp. RCC4572	2565274
Chloroparvula sp. RCC696	2565275
Chloroparvula sp. RCC999	2565276
Chloropicon laureae Lopes dos Santos & Eikrem, 2017	464258
Chloropicon mariensis Eikrem & Lopes dos Santos, 2017	1606511


# Find matches with gndiff

Name matching between scientific names (including authors) from GBIF and the NCBI Taxonomy (subset of taxa belonging to Viridiplantae), with authors where available.

[Gndiff](https://github.com/gnames/gndiff) is a tool from the Global Names Architecture project, which allows comparison of Linnean scientific names between two text tables. There are other GNA tools available, including a lookup service (Gnfinder) but we decided on Gndiff because we wanted a tool that could be used offline with a specific version of the databases of interest, in order to keep analysis reproducible.

In [77]:
%%bash
sed -i 's/\"//g' ncbi_viridiplantae.tsv # Strip quote chars
gndiff --format pretty dataset_merged_curr.tsv ncbi_viridiplantae.tsv > round_1.json

In [78]:
def classify_gndiff_result(match):
    """Classify name matching results from gndiff
    
    Names from GBIF used as source, names NCBI taxonomy as reference. Gndiff performs name
    matching (including author if available) and classifies matches as "Exact", "Partial",
    or "Fuzzy". However, the "Exact" match only takes the binomen into account, so the
    author fields may be missing or mismatched. We wish to be able to detect homonyms so
    the author field is important to us.
    
    Parameters
    ----------
    match : dict
        Individual match results in list 'Matches' from gndiff results. Results reported
        in json format and parsed by python json.load
    
    Returns
    -------
    dict
        With the following keys: `gbif_speciesName`, `gbif_taxonID` (from `sourceRecord`),
        `ncbi_speciesName`, `ncbi_taxonID` (from `referenceRecords`), and the hit
        classified under `status` (possible values: `exact_match`, `author_mismatch`, `noauthor`,
        `fuzzy`, `partial`, `no_hit`.
    """
    rec = {
        'gbif_speciesName' : i['sourceRecord']['name'],
        'gbif_taxonID' : i['sourceRecord']['id'],
    }
    if 'referenceRecords' in match and match['referenceRecords']: # key present and not null
        # At least one hit - if multiple hits, top hit should have best score
        if len(match['referenceRecords']) >= 1:
            # Record match in NCBI database
            rec['ncbi_speciesName'] = match['referenceRecords'][0]['name']
            rec['ncbi_taxonID'] = match['referenceRecords'][0]['id']
            # Exact matching names
            if match['referenceRecords'][0]['matchType'] == 'Exact':
                # Author names are present
                if 'authors' in match['sourceRecord'] and 'authors' in match['referenceRecords'][0]:
                    # Author names exactly match
                    if set(match['sourceRecord']['authors']) == set(match['referenceRecords'][0]['authors']):
                        rec['status'] = 'exact_match'
                    # Author names do not match
                    else:
                        rec['status'] = 'author_mismatch'
                # One or both sets of author names absent
                else:
                    rec['status'] = 'noauthor'
            elif match['referenceRecords'][0]['matchType'] == 'Fuzzy':
                # Fuzzy match
                rec['status'] = 'fuzzy'
            elif match['referenceRecords'][0]['matchType'].startswith("Partial"):
                # If best hit is only a partial match, e.g. only genus matches,
                # then it is discarded
                rec['status'] = 'partial'
                rec['ncbi_speciesName'] = ""
                rec['ncbi_taxonID'] = ""
    else:
        rec['status'] = 'no_hit'
    return(rec)

In [79]:
# Read results from gndiff
with open('round_1.json', 'r') as fh:
    hits1 = json.load(fh)

# Classify status of results by our criteria
hits1_parsed = []
for i in hits1['Matches']:
    rec = classify_gndiff_result(i)
    hits1_parsed.append(rec)

# Convert to data frame
hits1_df = pd.DataFrame(hits1_parsed).fillna('')

In [80]:
hits1_df.value_counts("status", dropna=False)

status
exact_match        3526
partial            1441
author_mismatch     445
no_hit              207
noauthor             67
fuzzy                35
Name: count, dtype: int64

In [81]:
# Merge matched names with the original table
round1_matched = pd.merge(
    left=dataset_merged_curr,
    right=hits1_df.query("ncbi_taxonID != ''"),
    how='right',
    left_on="taxonID",
    right_on="gbif_taxonID",
)[[
    'dataset_scientificName',
    'dataset_taxonID',
    'acceptedNameUsageID',
    'acceptedName',
    'gbif_speciesName',
    'gbif_taxonID',
    'taxonomicStatus',
    'ncbi_speciesName',
    'ncbi_taxonID',
    'status'
]]

In [82]:
round1_matched.head()

,dataset_scientificName,dataset_taxonID,acceptedNameUsageID,acceptedName,gbif_speciesName,gbif_taxonID,taxonomicStatus,ncbi_speciesName,ncbi_taxonID,status
0,Cucumis sativus L.,2874569,2874569,Cucumis sativus L.,Cucumis sativus L.,2874569,accepted,Cucumis sativus L.,3659,exact_match
1,Pyrola media Sw.,2888292,2888292,Pyrola media Sw.,Pyrola media Sw.,2888292,accepted,Pyrola media Sw.,642534,exact_match
2,Hedera colchica (K.Koch) K.Koch,3036033,3036033,Hedera colchica (K.Koch) K.Koch,Hedera colchica (K.Koch) K.Koch,3036033,accepted,Hedera colchica (K.Koch) K.Koch,82097,exact_match
3,Vinca minor L.,3169707,3169707,Vinca minor L.,Vinca minor L.,3169707,accepted,Vinca minor L.,60093,exact_match
4,Drosera intermedia Hayne,3191157,3191157,Drosera intermedia Hayne,Drosera intermedia Hayne,3191157,accepted,Drosera intermedia Hayne,463625,exact_match


In [83]:
round1_matched.shape[0]

4073

## For names with no hits, get synonyms and do matching again

Matches in the NCBI taxonomy were not found for about a fifth of the GBIF names in the first round. It is possible that some of those species are known under a different synonym in the NCBI database.

From the GBIF databsae, we search for known synonyms to each name by finding other names with the same `acceptedNameUsageID`, and then match these synonyms against the NCBI list.

Note that this assumes that the synonyms represent equivalent taxonomic concepts and that synonymy is transitive, which may not be true!

In [84]:
# Names with no hits in the first round of matching
round1_nohits = pd.merge(
    left=hits1_df.query("ncbi_taxonID == ''"),
    right=dataset_merged_curr,
    left_on="gbif_taxonID",
    right_on="taxonID"
)[['dataset_scientificName','dataset_taxonID','acceptedNameUsageID','acceptedName']]

In [85]:
# Get the synonyms from the GBIF Backbone Taxonomy
round1_nohits_synonyms = pd.merge(
    left=round1_nohits,
    right=gbif_backbone[['taxonID','scientificName','acceptedNameUsageID','taxonomicStatus']],
    left_on="acceptedNameUsageID",
    right_on="acceptedNameUsageID"
)

In [86]:
round1_nohits_synonyms.value_counts("taxonomicStatus", dropna=False)

taxonomicStatus
synonym                15891
homotypic synonym       1964
accepted                1640
heterotypic synonym     1640
proparte synonym          35
Name: count, dtype: int64

In [87]:
# Write to CSV for Gndiff
round1_nohits_synonyms[['scientificName','taxonID']].to_csv('synonyms.tsv', sep="\t", index=False, quoting=None)

In [88]:
%%bash
gndiff --format pretty synonyms.tsv ncbi_viridiplantae.tsv > round_2.json

In [89]:
# Read results from gndiff
with open('round_2.json', 'r') as fh:
    hits2 = json.load(fh)

# Classify status of results by our criteria
hits2_parsed = []
for i in hits2['Matches']:
    rec = classify_gndiff_result(i)
    hits2_parsed.append(rec)

# Convert to data frame
hits2_df = pd.DataFrame(hits2_parsed).fillna('')

In [90]:
hits2_df.shape

(21131, 5)

In [91]:
# Majority of synonyms do not have a hit
hits2_df.query('ncbi_taxonID == ""').shape

(20323, 5)

In [92]:
hits2_df.query('ncbi_taxonID != ""')

,gbif_speciesName,gbif_taxonID,ncbi_speciesName,ncbi_taxonID,status
14,Potentilla humifusa (Fr.) Zimmeter,7853059,Potentilla humifusa Willd.,1819534,author_mismatch
55,Rubus rugosus G.Jens.,8639283,Rubus rugosus,2487660,noauthor
56,Rubus rugosus G.Jens. ex Lange,7726759,Rubus rugosus,2487660,noauthor
61,Salix matsudana var. tortuosa Vilm.,8309320,Salix matsudana var. tortuosa Vilm.,349989,exact_match
69,Salix matsudana f. tortuosa (Vilm.) Rehder,6711149,Salix matsudana var. tortuosa Vilm.,349989,author_mismatch
...,...,...,...,...,...
20967,Peucedanum sulcatum Nyman,7588748,Peucedanum sulcatum Sond.,408988,author_mismatch
21066,Schkuhria pinnata (Lam.) Kuntze,8346536,Schkuhria pinnata (Lam.) Kuntze ex Thell.,176579,author_mismatch
21070,"Schkuhria pinnata Kuntze ex Heiser, 1945",8208789,Schkuhria pinnata (Lam.) Kuntze ex Thell.,176579,author_mismatch
21094,Schkuhria pinnata (Lam.) Cabrera,7752372,Schkuhria pinnata (Lam.) Kuntze ex Thell.,176579,author_mismatch


In [93]:
# Get synonyms with hits from the second round of matching
round2_matched = pd.merge(
    left=round1_nohits_synonyms,
    right=hits2_df.query('ncbi_taxonID != ""'),
    how='right',
    left_on="taxonID",
    right_on="gbif_taxonID"
)[[
    'dataset_scientificName',
    'dataset_taxonID',
    'acceptedNameUsageID',
    'acceptedName',
    'gbif_speciesName',
    'gbif_taxonID',
    'taxonomicStatus',
    'ncbi_speciesName',
    'ncbi_taxonID',
    'status'
]]

In [97]:
round1_matched.shape

(4073, 10)

In [98]:
round2_matched.shape

(906, 10)

In [115]:
# Merge the original checklist names and IDs with the matched NCBI names
dataset_merged_upd = pd.merge(
    left=dataset_merged_curr[['dataset_scientificName','dataset_taxonID','acceptedNameUsageID','acceptedName']],
    right=pd.concat([round1_matched,round2_matched]),
    left_on=['dataset_scientificName','dataset_taxonID','acceptedNameUsageID','acceptedName'],
    right_on=['dataset_scientificName','dataset_taxonID','acceptedNameUsageID','acceptedName'],
    how='left'
).fillna('')

In [116]:
# Separate into those with and without hits
dataset_merged_matched = dataset_merged_upd[dataset_merged_upd['gbif_speciesName'] != ""]
dataset_merged_unmatched = dataset_merged_upd[dataset_merged_upd['gbif_speciesName'] == ""]

In [141]:
# For those species without hits,
# Fill in gbif_speciesName and taxonomicStatus with the current values corresponding to this taxonID in the GBIF backbone
dataset_merged_unmatched = pd.merge(
    left=gbif_backbone[['taxonID','scientificName','taxonomicStatus']].rename(columns={'taxonID' : 'gbif_taxonID', 'scientificName' : 'gbif_speciesName'}),
    right=dataset_merged_unmatched.drop(['gbif_speciesName', 'gbif_taxonID', 'taxonomicStatus'],axis=1),
    left_on="gbif_taxonID",
    right_on="dataset_taxonID",
    how="right",
)[[
    'dataset_scientificName',
    'dataset_taxonID',
    'acceptedNameUsageID',
    'acceptedName',
    'gbif_speciesName',
    'gbif_taxonID',
    'taxonomicStatus',
    'ncbi_speciesName',
    'ncbi_taxonID',
    'status'
]]

In [144]:
# Combine again into a single table
dataset_merged_upd = pd.concat([dataset_merged_matched,dataset_merged_unmatched]).sort_values('status')